In [1]:
import folium
import geopandas as gpd

In [2]:
# Carregando os dados
br = gpd.read_file("BR_Pais_2022/BR_Pais_2022.shp")
ti = gpd.read_file("tis_poligonais/tis_poligonais.shp")
data = gpd.read_file("escolas_2023_coordenadas.geojson")  

/opt/anaconda3/lib/python3.11/site-packages/pyogrio/raw.py:196: RuntimeWarning: tis_poligonais/tis_poligonais.shp contains polygon(s) with rings with invalid winding order. Autocorrecting them, but that shapefile should be corrected using ogr2ogr for example.
  return ogr_read(


In [3]:

# Creating map as "m"
m = folium.Map(location=[-15.7801, -47.9292], zoom_start=4)

# Adding Brazil's border
folium.GeoJson(
    br,
    name="Brazil",
    style_function=lambda x: {'color': 'black', 'weight': 0.5}
).add_to(m)

# Adding indigenous lands
folium.GeoJson(
    ti,
    name="Indigenous lands",
    style_function=lambda x: {'color': 'grey', 'fillOpacity': 0.5}
).add_to(m)

# Adding schools with tooltips
for _, row in data.iterrows():
    folium.CircleMarker(
        location=[row.geometry.y, row.geometry.x],
        radius=2,
        color='blue' if row['IN_INTERNET'] == 1 else 'red',
        fill=True,
        fill_opacity=0.6,
        tooltip=f"School: {row['NO_ENTIDADE']} <br> Internet: {'Yes' if row['IN_AGUA_POTAVEL'] == 1 else 'No'}"
    ).add_to(m)

# Adding legend with html/css
legend_html = '''
<div style="position: fixed; 
    bottom: 50px; left: 50px; width: 120px; height: 70px; 
    background-color: white; border:2px solid grey; z-index:9999; font-size:14px;
    ">
    &nbsp;<b>Internet: </b><br>
    &nbsp;<i class="fa fa-circle" style="color:red"></i>&nbsp;No<br>
    &nbsp;<i class="fa fa-circle" style="color:blue"></i>&nbsp;Yes<br>
</div>
'''

github = '''
<div style="position: fixed; 
    top: 50px; left: 50px; width: 120px; height: 70px; 
    z-index:9999; font-size:14px;
    ">
    &nbsp;<b><a href="https://github.com/giovannacbs/brazilschool-maps" target="_blank" style="text-decoration:none; color:black;">Project repertory on Github</a></b>
</div>
'''

m.get_root().html.add_child(folium.Element(legend_html))
m.get_root().html.add_child(folium.Element(github))


# Saving as html
m.save('schools2023/internet2023.html')
